# Density Functional Theory 2 

In this set of exercises, you will assess the performance of various
state-of-the-art density functionals in the prediction of geometric
properties, and you will again compare your results to both wavefunction
theory and experimental data. 

## Structural Parameters of NO$_3\cdot$: Performance of DFT, HF and MP2

Before diving into the theoretical aspects of exchange-correlation
functionals, you will put a representative selection of functionals to
good use in a notoriously tricky system, the NO$_3\cdot$ radical.
Nitrite radicals are highly reactive species that are rapidly destroyed
by sunlight, but as the sun sets, they start to play an important role
in chemical transformations (in what is called the night-time chemistry
of the atmosphere) [^radical]. 

There exist various experimental and theoretical
studies of NO$_3\cdot$ , with the experiment indicating a fully symmetric D$_3^h$
structure with equal N-O bond lengths of 1.24 Å and O-N-O bond angles of
120$^\circ$. 

In this exercise, you will be comparing the performance of
various DFT exchange-correlation functionals, Hartree-Fock theory and
MP2 in predicting these structural parameters.

[^radical]: *Phys. Chem. Chem. Phys.*, **2014**, 16, *19437*

First, we'll import the required modules.

In [1]:
import psi4

import sys
sys.path.append("..")


psi4.set_memory('2 GB')
psi4.set_num_threads(2)



In [2]:
from helpers import *

This is our starting geometry.

In [4]:
radical = psi4.geometry("""
symmetry c1
0 2
 N     0.000000     0.000000     0.000000
 O     1.400000     0.000000     0.000000
 O    -1.000000     1.000000     0.000000
 O    -1.000000    -1.000000     0.000000
""")



## HF

First, we'll do a hartree fock calculation. As this is a difficult system we make use of a second order SCF convergence method implemented in Psi4. 

SOSCF makes use of the fact that the energy that we want to minimize in the SCF calculation has a gradient (and a hessian) both with respect to geometry changes and orbital changes. 
In a standard SCF calculation as you get close to convergence the orbital changes will be very small.  When using SOSCF, Psi4 uses the orbital hessian to make the next orbital step when performing the SCF calculation. In the code, the energy is written as a taylor expansion of orbitals and truncated for all terms higher than second order. This helps in converging tricky systems but requires that one is already close to a minimum. So by setting `soscf` to `true` by default the soscf procedure is switched on when the gradient RMS is below $1.0 \times 10^{-2}$. By specifying `soscf_max_iter` we increase the number of SOSCF steps from the default of 5 to 20. 

In [ ]:
method = "hf"

psi4.core.set_output_file(f'radical_opt_{method}.log', False)
psi4.set_options({'reference':'uhf','guess':'gwh', 'soscf':True, 'soscf_max_iter':20,'maxiter':200})
E_hf = psi4.optimize('hf/6-31+G*', molecule=radical)

In [ ]:
E_hf

## MP2

In [ ]:
method = "mp2"


psi4.core.clean_options()
psi4.core.set_output_file(f'radical_opt_{method}.log', False)
psi4.set_options({'reference':'uhf','guess':'read', 'mp2_type':'conv', 'maxiter':200, 'soscf':True, 'soscf_max_iter':20})
E_mp2, wfn = psi4.optimize(f'{method}/6-31+G*', molecule=radical, return_wfn=True)

In [ ]:
E_mp2

# Visualizing the orbitals

In order to assess the reliability of an electronic structure
calculation, it is usually not sufficient to simply glance at the
optimised structure and energies. One should as well examine the
orbitals. This is also valid for DFT: Although the Kohn-Sham orbitals
lack, in a strict sense, any physical interpretation, experience shows
that they are *very close* to wavefunction-based single particle
orbitals (*cf.* chapter 5.3). Thus, the DFT Kohn-Sham orbitals provide a
useful interpretative tool to visualise changes in the electronic
structure between different species (or in a chemical transformation),
but one should bear in mind that they have to be interpreted with care.

Refer to section {ref}`interpretation-orbital` for more details

In Psi4 we will use the `frontier_orbitals` setting, which produces `cube` representations of the frontier molecular orbitals. For closed shell species, the highest occupied (HOMO) and the lowest unoccupied (LUMO) alpha orbitals (ie. $\psi_\alpha(r)$) are printed, while for open shell species a total of ($4+M_s$) orbitals are printed ($\alpha$ and $\beta$ spin for both lowest virtual (LVMO) and highest doubly occupied orbitals (DOMO), along with all singly occupied (SOMO) orbitals).
The plot below will print $\alpha$ spin orbitals in red and $\beta$ spin orbitals in blue.

Unfortuneately, there is a bug in psi4 1.4rc2 when printing cube files for MP2 wavefunctions. We therefore provide you with the necessary cube files which you can visualize using the commands `findCubeFiles(pathToDirectory)` and `showOrbitals(pathToDirectory)`

Using cubeprop you can also acces the electron density (`density`), the basis functions (`basis_functions`), the electrostatic potential (`esp`) and the a descriptor calculated from the frontier orbitals which is $f^2(r) = \rho_{\text{LUMO}}(r) - \rho_{\text{HOMO}}(r)$ (`dual_descriptor`). This descriptor is a good measure of nucleophilicity and electrophilicity and is like both Fukui functions combined.

In [9]:
# This is the code to get the cube files printed, 
# Due to a bug in Psi4 for mp2 those files are provided to you in the mp2-cube folder.

#psi4.set_options({'cubeprop_tasks':['frontier_orbitals']})
#psi4.cubeprop(wfn)

In [10]:
orbitals = findCubeFiles('./mp2-cube/')
show_orbitals(wd='./mp2-cube/', mol=radical)

interactive(children=(Dropdown(description='Orbital:', index=1, options=('LVMO', 'SOMO', 'DOMO'), value='SOMO'…

In [11]:
drawXYZ_labeled(radical)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
# Convert molecule coordinates to numpy array
r_numpy = radical.to_arrays()[0]*hartree2A 

print('MP2 values')
print(f'r1: {calculate_bond(r_numpy[0], r_numpy[1]):.3f}')
print(f'r2: {calculate_bond(r_numpy[0], r_numpy[3]):.3f}')
print(f'r3: {calculate_bond(r_numpy[0], r_numpy[2]):.3f}')
print(f'O-N-O angle: {calculate_angle(r_numpy[1], r_numpy[0], r_numpy[2]):.3f}')
print(f'O-N-O angle: {calculate_angle(r_numpy[3], r_numpy[0], r_numpy[2]):.3f}')

## DFT

Run the same geometry optimization for two of the following density functionals (where the keyword differs from the functional abbreviation, it is given in parentheses): LDA (SVWN5), BLYP, BP86, PBE, B3LYP, B97-2, M06-L, M06-2X, TPSS, mPW1PW91, wb97x-d. 

Your assistants will assign to you which functionals to use. For this small system, Hartree-Fock and MP2 will be surprisingly fast; however, you may rest assured that for larger molecules and basis sets, all the DFT methods will outperform MP2 in computational efficiency, with some of them even beating Hartree-Fock

In [ ]:
psi4.core.clean_options()
psi4.core.set_output_file(f'radical_opt_{method}.log', False)
psi4.set_options({'reference':''}) # Choose a reference here, for DFT it can be Unrestricted KS (uks), restricted (rks) or restricted open shell (roks)
E_dft, wfn = psi4.optimize(f'method/basis', molecule=radical, return_wfn=True)

In [14]:
psi4.set_options({'cubeprop_tasks':['frontier_orbitals']})
psi4.cubeprop(wfn)

In [15]:
orbitals = findCubeFiles() # if we provide no path, the function looks in the current directory for the cube files
show_orbitals(mol=radical)

interactive(children=(Dropdown(description='Orbital:', index=1, options=('LVMO', 'SOMO', 'DOMO'), value='SOMO'…

In [ ]:
# Convert molecule coordinates to numpy array
r_numpy = radical.to_arrays()[0]*hartree2A 

print('DFT: B3LYP')
print(f'r1: {calculate_bond(r_numpy[0], r_numpy[1]):.3f}')
print(f'r2: {calculate_bond(r_numpy[0], r_numpy[3]):.3f}')
print(f'r3: {calculate_bond(r_numpy[0], r_numpy[2]):.3f}')
print(f'O-N-O angle: {calculate_angle(r_numpy[1], r_numpy[0], r_numpy[2]):.3f}')
print(f'O-N-O angle: {calculate_angle(r_numpy[3], r_numpy[0], r_numpy[2]):.3f}')

|Method:   |  $\phi_1$ \[$^\circ$\] |  $\phi_2$ \[$^\circ$\] |  $\mathbf{r}_1 (O-N)$ \[Å\] |  $\mathbf{r}_2 (O-N)$ \[Å\] |  $\mathbf{r}_3 (O-N)$ \[Å\] |  Symmetry |
|----------| ----------------------- | ----------------------- | ---------------------------- |---------------------------- | ---------------------------- | ----------|
| Exp.       |          120         |            120         |              1.24    |          1.24                         |1.24       |       D$_3^h$ |

 ```{admonition} Exercise 
:class: exercise
    Compare the performance of the methods in predicting accurate
    structures. For the DFT methods, specify what approximations are
    used in the exchange functional (you may refer to {ref}`dfttheory`). Is there a trend relating the complexity
    of the exchange-correlation functional (LDA, GGA, hybrid, ...) to
    the quality of your results?
```

```{admonition} Exercise
:class: exercise
Take a screenshot of the MP2 SOMO and include it in your report. Take a screenshot of one of the DFT SOMOs as well.

How do the orbitals differ from each other? How does the symmetry of the SOMO relate to the predicted structure?
```

 ```{admonition} Exercise
:class: exercise
Explain the difference between static and dynamic correlation. Relate this to the different results you obtained for the nitrate radical. What kind of correlation do HF, MP2 and DFT take into account?
```

```{admonition} Exercise
:class: exercise
If you needed a highly accurate structure and energy for  NO$_3\cdot$, which method would you use?
```